# Music Genre Classification

In [ ]:
!pip install pretty_midi

In [ ]:
!pip install numpy
!pip install pandas
!pip install pip install scikit-learn

In [ ]:
# Credit: Code adapted and used from Sander Shi's Colab

## Download and Parse Genre Labels
* Go to the website http://www.tagtraum.com/msd_genre_datasets.html.
* Look for the section labeled "CD1" and download the associated zip file.
* Once the download is complete, unzip the file


In [ ]:
# Unzips the "CD1" zip file
!unzip msd_tagtraum_cd1.cls.zip

In [1]:
import pandas as pd

def get_genres(path):
    """
    Stores the genre labels into a pandas data frame.
    
    Parameters:
        path (str): The path to the genre label file.
        
    Returns:
        pandas.DataFrame: A data frame containing the genres and MIDI IDs.
    """
    ids = []
    genres = []
    
    with open(path) as f:
        for line in f:
            # Skip lines starting with '#'
            if not line.startswith('#'):
                # Splits the line by the tab character ('\t') and unpacks the resulting values 
                # into variables x and y. The strip() function removes leading and trailing whitespace from the line.
                x, y, *_ = line.strip().split("\t")
                # Appends the value of x (track ID) to the ids list.
                ids.append(x)
                # Appends the value of y (genre) to the genres list.
                genres.append(y)
    
    # Constructs a data frame with two columns, "Genre" and "TrackID", using a dictionary. 
    # The "Genre" column contains the genres stored in the genres list, and the "TrackID" column 
    # contains the track IDs stored in the ids list.
    genre_df = pd.DataFrame(data={"Genre": genres, "TrackID": ids})
    return genre_df

# genre_path: path of the unzipped "CD1" file
genre_path = "msd_tagtraum_cd1.cls"
# creates the genres data frame
genre_df = get_genres(genre_path)

# Get unique genre labels
label_list = list(set(genre_df.Genre))

# Create a dictionary mapping genre labels to their index
label_dict = {lbl: label_list.index(lbl) for lbl in label_list}

print(genre_df.head(), end="\n\n")  # Display the first few rows of the genre data frame
print(label_list, end="\n\n")  # Display the unique genre labels
print(label_dict, end="\n\n")  # Display the genre dictionary mapping labels to indices

      Genre             TrackID
0  Pop_Rock  TRAAAAK128F9318786
1       Rap  TRAAAAW128F429D538
2  Pop_Rock  TRAAABD128F429CF47
3      Jazz  TRAAAED128E0783FAB
4  Pop_Rock  TRAAAEF128F4273421

['Folk', 'Pop_Rock', 'International', 'Electronic', 'Jazz', 'Country', 'Latin', 'New Age', 'RnB', 'Reggae', 'Vocal', 'Blues', 'Rap']

{'Folk': 0, 'Pop_Rock': 1, 'International': 2, 'Electronic': 3, 'Jazz': 4, 'Country': 5, 'Latin': 6, 'New Age': 7, 'RnB': 8, 'Reggae': 9, 'Vocal': 10, 'Blues': 11, 'Rap': 12}



In [5]:
#cd /content/drive/MyDrive

## Download, Parse and Match Midi Files
* Visit the website http://colinraffel.com/projects/lmd/.
* Find the section titled "LMD-matched" and click on the provided link. This will initiate the download of a MIDI dataset where each file is matched to an entry in the million song dataset.
* Once the download is complete, untar the file.

In [ ]:
# Untars the downloaded file
!tar -xvf lmd_matched.tar.gz

In [2]:
import os
import pandas as pd
import pickle

def get_matched_midi(midi_folder, genre_df):
    """
    Loads the MIDI file paths from the given folder and creates a pandas DataFrame.
    Matches each MIDI file with a genre based on the genre_df generated by get_genres.

    Parameters:
        midi_folder (str): The path to the MIDI files folder.
        genre_df (pandas.DataFrame): The genre label DataFrame generated by get_genres.

    Returns:
        pandas.DataFrame: A DataFrame containing the track ID and the path to the corresponding MIDI file.
    """
    # Get all MIDI files
    track_ids = []
    file_paths = []

    for dir_name, subdir_list, file_list in os.walk(midi_folder):
        # Checks if the length of the directory name is equal to 36. 
        if len(dir_name) == 36:
            # Extracts the track ID from the directory name by slicing the string from index 18 to the end
            track_id = dir_name[18:]
            # Constructs the file path by joining the directory name and the first file in the file_list
            # ***(assumes that each directory contains only one MIDI file)*** 
            # ATTENTION: some directories actually contain more than one MIDI file for that genre
            # file_list[0] is done for Simplification sake
            file_path = os.path.join(dir_name, file_list[0])
            track_ids.append(track_id)
            file_paths.append(file_path)

    # Constructs a DataFrame with two columns, "TrackID" and "Path", using a dictionary. 
    # The "TrackID" column contains the track IDs stored in the track_ids list, and 
    # The "Path" column contains the file paths stored in the file_paths list
    all_midi_df = pd.DataFrame({"TrackID": track_ids, "Path": file_paths})

    # Inner join the frames with the genre DataFrame
    df = pd.merge(all_midi_df, genre_df, on='TrackID', how='inner')
    
    # Drop the redundant TrackID column
    df = df.drop(["TrackID"], axis=1)

    return df

# midi_path: path to lmd_matched folder created from running previous tar command
midi_path = "lmd_matched"
# matched_midi_df: data frame with matched genres to file paths
matched_midi_df = get_matched_midi(midi_path, genre_df)

# Optionally save the matched_midi_df DataFrame as a pickle file
with open("matched_midi.pkl", "wb") as f:
    pickle.dump(matched_midi_df, f)

# Print the first few rows
print(matched_midi_df.head())

                                                Path     Genre
0  lmd_matched\A\A\A\TRAAAGR128F425B14B\1d9d16a9d...  Pop_Rock
1  lmd_matched\A\A\D\TRAADKW128E079503A\3797e9b9a...  Pop_Rock
2  lmd_matched\A\A\F\TRAAFMT128F429DB58\0a4f2051b...  Pop_Rock
3  lmd_matched\A\A\G\TRAAGMC128F4292D0F\0644195d1...   Country
4  lmd_matched\A\A\L\TRAALAH128E078234A\8cfecf566...  Pop_Rock


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pickle

# If saved, load matched_midi_df from the saved pickle file
with open('matched_midi.pkl', 'rb') as f:
    matched_midi_df = pickle.load(f)

print(matched_midi_df.head())

                                                Path     Genre
0  lmd_matched\A\A\A\TRAAAGR128F425B14B\1d9d16a9d...  Pop_Rock
1  lmd_matched\A\A\D\TRAADKW128E079503A\3797e9b9a...  Pop_Rock
2  lmd_matched\A\A\F\TRAAFMT128F429DB58\0a4f2051b...  Pop_Rock
3  lmd_matched\A\A\G\TRAAGMC128F4292D0F\0644195d1...   Country
4  lmd_matched\A\A\L\TRAALAH128E078234A\8cfecf566...  Pop_Rock


## Extract Midi Files Features

* Ensure you have the necessary libraries installed, such as pretty_midi, numpy, librosa, fluidsynth, and pickle.

In [4]:
# Library needed for feature extraction
!pip install pyfluidsynth


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\jkamp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
# import librosa
# import fluidsynth
import pickle
import numpy as np
import pretty_midi
import warnings
from sklearn.utils import resample
  

def normalize_features(features):
    """
    Normalizes the features to the range [-1, 1].

    Parameters:
        features (list of float): The array of features.

    Returns:
        list of float: Normalized features.
    """
    # Normalize each feature based on its specific range
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    melody_complexity = (features[5] - 0) / 10
    melody_range = (features[6] - 0) / 80

    # Normalize pitch class histogram
    pitch_class_hist = [((f - 0) / 100) for f in features[7:-1]]

    # Return the normalized feature vector
    return [tempo, resolution, time_sig_1, time_sig_2, melody_complexity, melody_range] + pitch_class_hist



def get_features(path):
    """
    Extracts specific features from a MIDI file given its path using the pretty_midi library.
    Handle any potential errors with MIDI files appropriately.

    Parameters:
        path (str): The path to the MIDI file.

    Returns:
        list of float: The extracted features.
    """
    try:
        # Checking for MIDI files
        with warnings.catch_warnings():
            warnings.simplefilter("error")

            # Creates a PrettyMIDI object by loading the MIDI file specified by the given path.
            file = pretty_midi.PrettyMIDI(path)
            
            # tempo: the estimated tempo of the audio file
            tempo = file.estimate_tempo()

            # num_sig_changes: the number of time signature changes in the audio file
            num_sig_changes = len(file.time_signature_changes)

            # resolution: the time resolution of the audio file (in ticks per beat)
            resolution = file.resolution


            # Extract time signature information
            ts_changes = file.time_signature_changes
            ts_1, ts_2 = 4, 4
            if len(ts_changes) > 0:
                ts_1 = ts_changes[0].numerator
                ts_2 = ts_changes[0].denominator
            
            # Extract melody-related features
            # melody: a pitch class histogram of the audio file
            melody = file.get_pitch_class_histogram()
            # melody_complexity: the number of unique pitch classes in the melody
            melody_complexity = np.sum(melody > 0)
            # melody_range: the range of pitch classes in the melody
            melody_range = np.max(melody) - np.min(melody)
            # OPTIONAL feature melody_contour: the temporal evolution of pitch content in the audio file
            # melody_contour = librosa.feature.tempogram(y=file.fluidsynth(fs=16000), sr=16000, hop_length=512)
            # melody_contour = np.mean(melody_contour, axis=0)
            # chroma: a chroma representation of the audio file
            chroma = file.get_chroma()
            # pitch_class_hist: the sum of the chroma matrix along the pitch axis
            pitch_class_hist = np.sum(chroma, axis=1)

            return normalize_features([tempo, num_sig_changes, resolution, ts_1,
                            ts_2, melody_complexity, melody_range] + list(pitch_class_hist)) # + list(melody_contour))
            
    # Discard MIDI file if there is an error
    except:
        return None

            

def extract_midi_features(path_df, oversample=False, undersample=False):
    """
    Extracts features and labels from MIDI files listed in the path DataFrame and concatenates the
    features with their labels into a matrix.

    Since the dataset is inherently unbalanced in terms of genre distribution, oversampling and 
    undersampling can be used to achieve a more balanced representation of features for each genre.

    Parameters:
        path_df (pandas.DataFrame): A DataFrame with paths to MIDI files and their matched genre.
        oversample (bool): Whether or not to perform oversampling on the data. Defaults to False.
        undersample (bool): Whether or not to perform undersampling on the data. Defaults to False.

    Returns:
        numpy.ndarray: A matrix of features along with labels.
    """
    all_features = []  # List to store all extracted features
    max_count = 0  # Variable to track the maximum count of MIDI files in a genre

    # Iterate through each genre in label_dict
    for genre in label_dict.keys(): #  Genre is the string from label_dict
        genre_df = path_df.loc[path_df['Genre'] == genre]  # DataFrame containing MIDI files of the current genre
        genre_count = len(genre_df)  # Count of MIDI files in the current genre
        if genre_count > max_count:
            max_count = genre_count  # Update the maximum count if the current genre has more MIDI files

        features_list = []  # List to store features of MIDI files in the current genre
        for index, row in genre_df.iterrows():
            # Extract features from MIDI file
            features = get_features(row.Path)
            # Map genre label to a number
            genre = label_dict[row.Genre]
            if features is not None:
                # Append the genre label to the feature vector
                # (i.e., it concatenates the features with the labels into a matrix)
                features.append(genre)
                features_list.append(features)  # Append the feature vector to the list

        if oversample:
            # Resample the features to match the maximum count (oversampling)
            features_list = resample(features_list, replace=True, n_samples=max_count, random_state=42)
        elif undersample:
            # Resample the features to match the maximum count (undersampling)
            features_list = resample(features_list, replace=False, n_samples=max_count, random_state=42)

        all_features += features_list  # Append the features of the current genre to the overall list

    # Return the numpy array of all extracted features along with corresponding genres
    return np.array(all_features)


# Call the extract_midi_features function with the appropriate path DataFrame to extract the MIDI 
# file features and obtain the feature-label matrix
labeled_features = extract_midi_features(matched_midi_df)
# Print the labeled features
print(labeled_features)

# Optionally store the feature-label matrix as a pickle file for further use
with open('labeled_features.pkl', 'wb') as f:
    pickle.dump(labeled_features, f)

ImportError: Couldn't find the FluidSynth library.

## Partition Dataset into Training, Validation, and Testing

In [ ]:
from keras.utils import to_categorical

# Shuffle the features
labeled_features = np.random.permutation(labeled_features)

# Partition the Dataset into 3 Sets: Training, Validation, and Test
num = len(labeled_features)
# Calculate the number of samples for training data (60% of the dataset)
num_training = int(num * 0.6)
# Calculate the number of samples for validation data (20% of the dataset)
num_validation = int(num * 0.8)

# Extract the training data (60% of the labeled features)
training_data = labeled_features[:num_training]
# Extract the validation data (20% of the labeled features)
validation_data = labeled_features[num_training:num_validation]
# Extract the test data (remaining 20% of the labeled features)
test_data = labeled_features[num_validation:]


# Separate the features from the labels
num_cols = training_data.shape[1] - 1
# Extract features from the training data
training_features = training_data[:, :num_cols]
# Extract features from the validation data
validation_features = validation_data[:, :num_cols]
# Extract features from the test data
test_features = test_data[:, :num_cols]

# Format the features for this multi-class classification problem
num_classes = len(label_list)
# Extract labels from the training data and convert them to integers
training_labels = training_data[:, num_cols].astype(int)
# Extract labels from the validation data and convert them to integers
validation_labels = validation_data[:, num_cols].astype(int)
# Extract labels from the test data and convert them to integers
test_labels = test_data[:, num_cols].astype(int)

print(test_features[:10])  # Print the first 10 rows of test features
print(test_labels[:10])  # Print the first 10 test labels
print(to_categorical((test_labels)[:10]))  # Print the one-hot encoding of the first 10 test labels

[[ 2.33436608e-01 -5.00000000e-02  1.25000000e-01  1.25000000e-01
   1.10000000e+00  2.50545653e-03  3.24309925e+04  3.02250000e+01
   5.61119725e+04  1.09800000e+02  3.70535700e+04  3.39100000e+03
   2.90382000e+04  6.45764000e+04  2.33650000e+03  3.26045600e+04
   4.68375000e+01]
 [ 3.95454709e-01  5.50000000e-01  1.25000000e-01  1.25000000e-01
   1.20000000e+00  1.78347608e-03  1.48019000e+04  1.98571500e+04
   5.61610000e+03  1.32613800e+04  3.11939400e+04  1.55619200e+04
   1.53120600e+04  8.21883000e+03  2.04780000e+04  2.36144100e+04
   4.77265000e+03]
 [ 2.96971127e-01  5.50000000e-01 -1.25000000e-01  1.25000000e-01
   1.10000000e+00  2.03850143e-03  1.43411000e+04  7.25648000e+03
   1.03569900e+04  1.61912900e+04  0.00000000e+00  1.12157500e+04
   5.93757000e+03  1.35004500e+04  4.33746000e+03  7.60182000e+03
   2.24264700e+04]
 [ 1.18608659e-01 -3.50000000e-01  6.25000000e-01  6.25000000e-01
   1.20000000e+00  1.58796142e-03  9.22426873e+03  2.24757553e+04
   8.62437045e+03  

In [ ]:
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder  
from keras.utils import to_categorical
import pickle


# Define the model architecture
model = keras.Sequential([
    # First hidden layer with 256 units and ReLU activation
    keras.layers.Dense(256, input_shape=(training_features.shape[1],), activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Second hidden layer with 128 units and ReLU activation
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Third hidden layer with 64 units and ReLU activation
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),  # Dropout layer to prevent overfitting
    
    # Output layer with num_classes units and softmax activation for multi-class classification
    keras.layers.Dense(num_classes, activation='softmax')
])

# Print a summary of the model architecture
print(model.summary())


"""
optimizer="adam": The optimizer algorithm to use during training. 
Adam optimizer is chosen, which is a popular optimization algorithm known for its efficiency.

loss='categorical_crossentropy': The loss function used to measure the discrepancy between the 
predicted output and the true output labels. Categorical cross-entropy is suitable for
multi-class classification tasks.

metrics=['accuracy']: The metric(s) to be evaluated during training and testing. 
Accuracy is a commonly used metric to assess the model's performance.
"""

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Encode the training and validation labels using one-hot encoding
train_labels_encoded = to_categorical(training_labels)
val_labels_encoded = to_categorical(validation_labels)


"""
training_features, train_labels_encoded: Input features and corresponding labels for model training.

validation_features, val_labels_encoded: Validation set used to monitor the model's performance 
                                         during training.

batch_size=32: Number of samples per gradient update. Training data is divided into batches, 
               and the model's weights are updated after each batch.

epochs=50: Number of times the model will iterate over the entire training dataset.

callbacks: EarlyStopping to stop training if the validation loss does not improve for a certain 
           number of epochs, and ModelCheckpoint to save the best model based on validation loss.
"""
history = model.fit(training_features, train_labels_encoded, 
                    validation_data=(validation_features, val_labels_encoded),
                    batch_size=32, epochs=50, verbose=2,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
                               keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)])

# Save the entire model to an h5 file
model.save("my_model.h5")

# Optionally save the entire model as a pickle file
with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)


# Optionally load the pickled model from file
with open('my_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Use the loaded model for prediction
preds = model.predict(test_features)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_features, to_categorical(test_labels))
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               4608      
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 13)                8